## Combine a graph exploring widget with a JSON viewer when clicking on nodes.

In [ ]:
import json

import ipycytoscape
import ipywidgets as W
import networkx as nx
import rdflib
from IPython.display import JSON, display
from rdflib.namespace import RDFS

from ipyradiant import CytoscapeViewer, FileManager, PathLoader

#### First, we will set up a widget instance using the basic Star Wars example used in other examples.

We will create a small example graph for use in the visualization.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX hum: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (hum:1)  # Luke
            (hum:4)  # Vader
            (film:1) # A New Hope
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))

In [ ]:
# instantiate the widget
cv = CytoscapeViewer(graph=simple_graph)

#### Then, we will need a class that is composed of both the graph widget and the `JSON` viewer. We will use these in an `ipywidgets.VBox`

In [ ]:
class CombinedExample(W.VBox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.json_output = W.Output()
        self.graph_widget = kwargs["graph_widget"]
        self.children = [self.graph_widget, self.json_output]
        self.graph_widget.cytoscape_widget.on("node", "click", self.load_json)

    def load_json(self, node):
        data = node["data"]
        data.pop("_label", None)
        data.pop("_attrs", None)
        with self.json_output:
            self.json_output.clear_output()
            display(JSON(data))

#### We can now see that if you click on a node in the cytoscape graph, the `JSON` details of that node will be displayed below the graph.

In [ ]:
ce = CombinedExample(graph_widget=cv)
ce